<a href="https://colab.research.google.com/github/nmaffly/rhythmoji/blob/main/Rhythmoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from io import StringIO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



In [3]:

# Read CSV data into a DataFrame
csv_data = "/content/music_fashion_improved_full_dataset.csv"
df = pd.read_csv(csv_data)

# Remove leading/trailing whitespaces from column names
df.columns = df.columns.str.strip()

# Separate data and target columns
data_columns = ['Genre_1', 'Genre_2', 'Genre_3']
target_columns = ['Attire_1', 'Attire_2', 'Attire_3']


In [4]:
#Split 'Top Genres' into separate columns
print(df[data_columns + target_columns])

            Genre_1       Genre_2       Genre_3              Attire_1  \
0          Soul/R&B     Classical  Instrumental         velvet jacket   
1              Jazz  Instrumental       Country           linen shirt   
2             Blues          Jazz         Metal     harmonica pendant   
3              Rock         Other         Vocal          band t-shirt   
4              Rock      Acoustic         Vocal        leather jacket   
..              ...           ...           ...                   ...   
195             Pop       Country      Acoustic          skinny jeans   
196     Hip Hop/Rap  Instrumental       Ambient              sneakers   
197           Metal       Country      Soul/R&B          studded belt   
198  Easy Listening          Rock       Ambient  relaxed fit clothing   
199            Punk     Classical         Blues           punk jacket   

                     Attire_2                Attire_3  
0                 dress shirt        classical outfit  
1          

In [5]:
# Combine genre columns into a single text column
df['genres'] = df[['Genre_1', 'Genre_2', 'Genre_3']].apply(lambda row: ' '.join(row), axis=1)


In [6]:
# Create a MultiLabelBinarizer to one-hot encode genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genres'].apply(lambda x: set(x.split(', '))))

In [7]:
# Create a CountVectorizer to represent attire text data
vectorizer = CountVectorizer()
attire_encoded = vectorizer.fit_transform(df[['Attire_1', 'Attire_2', 'Attire_3']].apply(lambda row: ' '.join(row), axis=1))

In [8]:
#Combine encoded features into X
X = pd.concat([pd.DataFrame(genre_encoded, columns=mlb.classes_), pd.DataFrame(attire_encoded.toarray(), columns=vectorizer.get_feature_names_out())], axis=1)

# Target variable (attire)
y = df[['Attire_1', 'Attire_2', 'Attire_3']]

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


UnimplementedError: ignored